this notebook test different contractor performance 

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms

# Contractor Test perodic condition

In [24]:
D=4;L=H=W=5;B=10;
tn2D_shape_list = [[(D,D)]+[(D,D,D)]*(H-1)]+[[(D,D,D)]+[(D,D,D,D)]*(H-1)]*(W-1)

In [ ]:
path,sublist_list,outlist = get_best_path(tn2D_shape_list,store=path_recorder,type='sub')

In [21]:
import numpy as np
import torch
import tensornetwork as tn
import opt_einsum as oe
import os,json
from tensornetwork.network_components import get_all_nondangling,get_all_dangling
from tensornetwork.contractors.opt_einsum_paths.utils import get_subgraph_dangling,get_all_edges

In [29]:
equation   = "kab,kcdb,kefcg,kgha->khedf"
tensor_l   = [torch.randn(B,D,D) ,torch.randn(B,D,D,D),torch.randn(B,D,D,D,D),torch.randn(B,D,D,D)]
oe.contract_path(equation, *tensor_l)[0]
path = oe.contract_path(equation, *tensor_l)[0]
oe.contract(equation,*tensor_l,optimize=path).shape

torch.Size([10, 4, 4, 4, 4])

In [38]:
def rd_engine(*x,**kargs):
    x =  torch.randn(*x,device='cpu',**kargs)
    x/=  torch.norm(x).sqrt()
    x =  torch.autograd.Variable(x,requires_grad=True)
    return x

In [34]:
path_recorder={}

In [32]:
from models.model_utils import *

In [35]:
tn2D_shape_list = [[(D,D)]+[(D,D,D)]*(H-1)]+[[(D,D,D)]+[(D,D,D,D)]*(H-1)]*(W-1)
path,sublist_list,outlist = get_best_path(tn2D_shape_list,store=path_recorder,type='sub')

In [39]:
tensor_list     = [rd_engine(B,*l) for t in tn2D_shape_list for l in t]
operands=[]
for tensor,sublist in zip(tensor_list,sublist_list):
    operand = [tensor,[...,*sublist]]
    operands+=operand
operands+= [[...,*outlist]]

In [42]:
oe.contract(*operands,optimize=path).shape

torch.Size([10, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4])

#### test_the_TRG_on_even_number

In [23]:
# #%%writefile Contractor2D/test_the_TRG_on_even_number.py
# import torch
# from Contractor2D.utils import apply_SVD

# tensor = torch.randn(4,4,2,2,2,2)

# print(uniform_shape_tensor_contractor_tn(tensor))

# truncate= None
# lu_tensor = tensor[0::2,0::2]
# ld_tensor = tensor[0::2,1::2]
# ru_tensor = tensor[1::2,0::2]
# rd_tensor = tensor[1::2,1::2]
# lu_lu,lu_rd = apply_SVD(lu_tensor,left_bond=[0,1],right_bond=[2,3],truncate=truncate)# abk ,kcd
# rd_lu,rd_rd = apply_SVD(rd_tensor,left_bond=[0,1],right_bond=[2,3],truncate=truncate)# abk, kcd
# ld_ld,ld_ru = apply_SVD(ld_tensor,left_bond=[1,2],right_bond=[3,0],truncate=truncate)# bck, kda
# ru_ld,ru_ru = apply_SVD(ru_tensor,left_bond=[1,2],right_bond=[3,0],truncate=truncate)# bck, kda
# tensor1     = torch.einsum("whicd,whjac,whbak,whdbl->whijkl",
#                             lu_rd,
#                             ld_ru,
#                             rd_lu,
#                             ru_ld)
# tensor2     = torch.einsum("whicd,whjac,whbak,whdbl->whijkl",
#                            rd_rd,
#                            ru_ru.roll(-1,1),
#                            lu_lu.roll(shifts=(-1, -1), dims=(0, 1)),
#                            ld_ld.roll(-1,0))

# print(tensor1_and_tensor2_contractor_tn(tensor1,tensor2))

# #print(torch.einsum("abcd,cdab->",tensor1[0,0],tensor2[0,0]))
# left,right  = apply_SVD(tensor1,left_bond=[0,1],right_bond=[2,3],truncate=truncate)# ABK,KCD
# lower,uppe  = apply_SVD(tensor2,left_bond=[1,2],right_bond=[3,0],truncate=truncate)# BCK,KDA
# # new_tensor  = torch.einsum("whicd,whjac,whbak,whdbl->whijkl",
# #                            right,
# #                            uppe,
# #                            left.roll(shifts=-1,dims=0),
# #                            lower.roll(shifts=1,dims=1))
# new_tensor  = torch.einsum("whadi,whjba,whkcb,whdcl->whijkl",
#                            lower,
#                            right.roll(-1,1),
#                            uppe.roll(-1,1),
#                            left.roll(shifts=(-1, -1), dims=(0, 1))
#                            )
# print(uniform_shape_tensor_contractor_tn(new_tensor))

# torch.einsum("abn,nji,lkh,hdc,caw,wkj,ilm,mbd->",
#              lu_lu[0,0],lu_rd[0,0],
#              rd_lu[0,0],rd_rd[0,0],
#              ld_ld[0,0],ld_ru[0,0],
#              ru_ld[0,0],ru_ru[0,0])

# torch.einsum("abji,lkdc,jcak,dilb->",
#              torch.einsum("abn,nji->abji",lu_lu[0,0],lu_rd[0,0]),
#              torch.einsum("lkh,hdc->lkdc",rd_lu[0,0],rd_rd[0,0]),
#              torch.einsum("caw,wkj->jcak",ld_ld[0,0],ld_ru[0,0]),
#              torch.einsum("ilm,mbd->dilb",ru_ld[0,0],ru_ru[0,0]))

# torch.dist(torch.einsum("abn,nji->abji",lu_lu[0,0],lu_rd[0,0]),lu_tensor)

# torch.dist(torch.einsum("lkh,hdc->lkdc",rd_lu[0,0],rd_rd[0,0]),rd_tensor)

# torch.dist(torch.einsum("caw,wkj->jcak",ld_ld[0,0],ld_ru[0,0]),ld_tensor)

# torch.dist(torch.einsum("ilm,mbd->dilb",ru_ld[0,0],ru_ru[0,0]),ru_tensor)

# torch.dist(lu_tensor[0,0],tensor[0,0])

# torch.dist(rd_tensor[0,0],tensor[1,1])

# torch.dist(ld_tensor[0,0],tensor[0,1])

# torch.dist(ru_tensor[0,0],tensor[1,0])

# torch.einsum("abji,lkdc,jcak,dilb->",
#              torch.einsum("abn,nji->abji",lu_lu[0,0],lu_rd[0,0]),
#              torch.einsum("lkh,hdc->lkdc",rd_lu[0,0],rd_rd[0,0]),
#              torch.einsum("caw,wkj->jcak",ld_ld[0,0],ld_ru[0,0]),
#              torch.einsum("ilm,mbd->dilb",ru_ld[0,0],ru_ru[0,0]))

# torch.einsum("abcd,idjb,ckal,jlik->",lu_tensor[0,0],ld_tensor[0,0],ru_tensor[0,0],rd_tensor[0,0])

# torch.einsum("abcd,ciaj,ldkb,kjli->",lu_tensor[0,0],ld_tensor[0,0],ru_tensor[0,0],rd_tensor[0,0])

# tensor1 = tensor[0,0]
# tensor2 = tensor[0,1]
# tensor3 = tensor[1,0]
# tensor4 = tensor[1,1]
# torch.einsum("abcd,idjb,ckal,jlik->",tensor1,tensor2,tensor3,tensor4)

Writing Contractor2D/test_the_TRG_on_even_number.py


#### check_the_brdc_coding_on_even_number

In [24]:
# #%%writefile Contractor2D/check_the_brdc_coding_on_even_number.py
# import torch
# from Contractor2D.utils import apply_SVD

# tensor     = torch.randn(8,8,2,2,2,2)/2
# uniform_shape_tensor_contractor_tn(tensor)

# lu_lu_origin,lu_rd_origin = apply_SVD(tensor[0::2,0::2],left_bond=[0,1],right_bond=[2,3],truncate=None)# abk ,kcd
# rd_lu_origin,rd_rd_origin = apply_SVD(tensor[1::2,1::2],left_bond=[0,1],right_bond=[2,3],truncate=None)# abk, kcd
# ld_ld_origin,ld_ru_origin = apply_SVD(tensor[0::2,1::2],left_bond=[1,2],right_bond=[3,0],truncate=None)# bck, kda
# ru_ld_origin,ru_ru_origin = apply_SVD(tensor[1::2,0::2],left_bond=[1,2],right_bond=[3,0],truncate=None)# bck, kda

# lu_lu,lu_rd = apply_SVD(bulk_tensor[0::2,0::2],left_bond=[0,1],right_bond=[2,3],truncate=None)# abk ,kcd
# rd_lu,rd_rd = apply_SVD(bulk_tensor[1::2,1::2],left_bond=[0,1],right_bond=[2,3],truncate=None)# abk, kcd
# ld_ld,ld_ru = apply_SVD(bulk_tensor[0::2,1::2],left_bond=[1,2],right_bond=[3,0],truncate=None)# bck, kda
# ru_ld,ru_ru = apply_SVD(bulk_tensor[1::2,0::2],left_bond=[1,2],right_bond=[3,0],truncate=None)# bck, kda

# print(lu_lu.shape)
# print(rd_lu.shape)
# print(ld_ld.shape)
# print(ru_ld.shape)

# lu_lu_should = lu_lu_origin[:,:]    ;lu_rd_should =lu_rd_origin[:,:]
# rd_lu_should = rd_lu_origin[:-1,:-1];rd_rd_should =rd_rd_origin[:-1,:-1]
# ld_ld_should = ld_ld_origin[:,:-1]  ;ld_ru_should =ld_ru_origin[:,:-1]
# ru_ld_should = ru_ld_origin[:-1,:]  ;ru_ru_should =ru_ru_origin[:-1,:]

# print(lu_lu_should.shape)
# print(rd_lu_should.shape)
# print(ld_ld_should.shape)
# print(ru_ld_should.shape)

# print(torch.dist(lu_lu_should,lu_lu),torch.dist(lu_rd_should,lu_rd))
# print(torch.dist(rd_lu_should,rd_lu),torch.dist(rd_rd_should,rd_rd))
# print(torch.dist(ld_ld_should,ld_ld),torch.dist(ld_ru_should,ld_ru))
# print(torch.dist(ru_ld_should,ru_ld),torch.dist(ru_ru_should,ru_ru))

# # right_edge == bulk_tensor[-1]
# # down_edge  == bulk_tensor[:,-1]

# rd_lu_r,rd_rd_r = apply_SVD(right_edge[1::2],left_bond=[0,1],right_bond=[2,3],truncate=None)# abk, kcd
# ru_ld_r,ru_ru_r = apply_SVD(right_edge[0::2],left_bond=[1,2],right_bond=[3,0],truncate=None)# bck, kda
# rd_lu_d,rd_rd_d = apply_SVD( down_edge[1::2],left_bond=[0,1],right_bond=[2,3],truncate=None)# abk, kcd
# ld_ld_d,ld_ru_d = apply_SVD( down_edge[0::2],left_bond=[1,2],right_bond=[3,0],truncate=None)# bck, kda
# rd_lu_c,rd_rd_c = apply_SVD(corn_tensor     ,left_bond=[0,1],right_bond=[2,3],truncate=None)# bck, kda

# rd_lu_r_should=rd_lu_origin[-1,:-1];rd_rd_r_should=rd_rd_origin[-1,:-1]
# ru_ld_r_should=ru_ld_origin[-1,:]  ;ru_ru_r_should=ru_ru_origin[-1,:]
# rd_lu_d_should=rd_lu_origin[:-1,-1];rd_rd_d_should=rd_rd_origin[:-1,-1]
# ld_ld_d_should=ld_ld_origin[:,-1]  ;ld_ru_d_should=ld_ru_origin[:,-1]
# rd_lu_c_should=rd_lu_origin[ -1,-1];rd_rd_c_should=rd_rd_origin[ -1,-1]

# print(rd_lu_r_should.shape)
# print(ru_ld_r_should.shape)
# print(rd_lu_d_should.shape)
# print(ld_ld_d_should.shape)
# print(rd_lu_c_should.shape)

# print(torch.dist(rd_lu_r_should,rd_lu_r),torch.dist(rd_rd_r_should,rd_rd_r))
# print(torch.dist(ru_ld_r_should,ru_ld_r),torch.dist(ru_ru_r_should,ru_ru_r))
# print(torch.dist(rd_lu_d_should,rd_lu_d),torch.dist(rd_rd_d_should,rd_rd_d))
# print(torch.dist(ld_ld_d_should,ld_ld_d),torch.dist(ld_ru_d_should,ld_ru_d))
# print(torch.dist(rd_lu_c_should,rd_lu_c),torch.dist(rd_rd_c_should,rd_rd_c))

# tensor1     = torch.einsum("whicd,whjac,whbak,whdbl->whijkl",
#                                     lu_rd_origin,
#                                     ld_ru_origin,
#                                     rd_lu_origin,
#                                     ru_ld_origin)
# tensor2     = torch.einsum("whicd,whjac,whbak,whdbl->whijkl",
#                            rd_rd_origin,
#                            ru_ru_origin.roll(shifts=-1,dims=1),
#                            lu_lu_origin.roll(shifts=(-1, -1), dims=(0, 1)),
#                            ld_ld_origin.roll(shifts=-1,dims=0))

# bulk_tensor1_should= tensor1[:-1,:-1]
# rigt_tensor1_should= tensor1[-1,:-1]
# down_tensor1_should= tensor1[:-1,-1]
# corn_tensor1_should= tensor1[-1,-1]
# bulk_tensor2_should= tensor2[:-1,:-1]
# rigt_tensor2_should= tensor2[-1,:-1]
# down_tensor2_should= tensor2[:-1,-1]
# corn_tensor2_should= tensor2[-1,-1]

# print(bulk_tensor1_should.shape)
# print(rigt_tensor1_should.shape)  
# print(down_tensor1_should.shape)  
# print(corn_tensor1_should.shape)  
# print(bulk_tensor2_should.shape)  
# print(rigt_tensor2_should.shape)  
# print(down_tensor2_should.shape)  
# print(corn_tensor2_should.shape)  

# bulk_tensor1 = torch.einsum("whicd,whjac,whbak,whdbl->whijkl",lu_rd[:-1,:-1],ld_ru[:-1],rd_lu,ru_ld[:,:-1])
# rigt_tensor1 = torch.einsum("wicd,wjac,wbak,wdbl->wijkl",lu_rd[-1,:-1],ld_ru[-1]   ,rd_lu_r, ru_ld_r[:-1])
# down_tensor1 = torch.einsum("wicd,wjac,wbak,wdbl->wijkl",lu_rd[:-1,-1],ld_ru_d[:-1],rd_lu_d, ru_ld[:,-1])
# corn_tensor1 = torch.einsum(" icd, jac, bak, dbl-> ijkl",lu_rd[-1,-1] ,ld_ru_d[-1] ,rd_lu_c, ru_ld_r[-1])
# bulk_tensor2 = torch.einsum("whicd,whjac,whbak,whdbl->whijkl",rd_rd,ru_ru[:,1:],lu_lu[1:,1:],ld_ld[1:])
# rigt_tensor2 = torch.einsum("wicd,wjac,wbak,wdbl->wijkl",rd_rd_r, ru_ru_r[1:],lu_lu[0,1:], ld_ld[0])
# down_tensor2 = torch.einsum("wicd,wjac,wbak,wdbl->wijkl",rd_rd_d, ru_ru[:,0],lu_lu[1:,0], ld_ld_d[1:])
# corn_tensor2 = torch.einsum(" icd, jac, bak, dbl-> ijkl",rd_rd_c, ru_ru_r[0],lu_lu[0,0] , ld_ld_d[0])

# print(bulk_tensor1.shape)
# print(rigt_tensor1.shape)  
# print(down_tensor1.shape)  
# print(corn_tensor1.shape)  
# print(bulk_tensor2.shape)  
# print(rigt_tensor2.shape)  
# print(down_tensor2.shape)  
# print(corn_tensor2.shape)  

# print(torch.dist(bulk_tensor1,bulk_tensor1_should))
# print(torch.dist(rigt_tensor1,rigt_tensor1_should))  
# print(torch.dist(down_tensor1,down_tensor1_should))  
# print(torch.dist(corn_tensor1,corn_tensor1_should))  
# print(torch.dist(bulk_tensor2,bulk_tensor2_should))  
# print(torch.dist(rigt_tensor2,rigt_tensor2_should))  
# print(torch.dist(down_tensor2,down_tensor2_should))  
# print(torch.dist(corn_tensor2,corn_tensor2_should)) 

# computer_vie_tn(tensor)

# truncate = None
# bulk_left ,bulk_right= apply_SVD(bulk_tensor1,left_bond=[0,1],right_bond=[2,3],truncate=truncate)
# rigt_left ,rigt_right= apply_SVD(rigt_tensor1,left_bond=[0,1],right_bond=[2,3],truncate=truncate)
# down_left ,down_right= apply_SVD(down_tensor1,left_bond=[0,1],right_bond=[2,3],truncate=truncate)
# corn_left ,corn_right= apply_SVD(corn_tensor1,left_bond=[0,1],right_bond=[2,3],truncate=truncate)
# bulk_lower,bulk_uppe = apply_SVD(bulk_tensor2,left_bond=[1,2],right_bond=[3,0],truncate=truncate)# BCK,KDA
# rigt_lower,rigt_uppe = apply_SVD(rigt_tensor2,left_bond=[1,2],right_bond=[3,0],truncate=truncate)# BCK,KDA
# down_lower,down_uppe = apply_SVD(down_tensor2,left_bond=[1,2],right_bond=[3,0],truncate=truncate)# BCK,KDA
# corn_lower,corn_uppe = apply_SVD(corn_tensor2,left_bond=[1,2],right_bond=[3,0],truncate=truncate)# BCK,KDA

# bulk_tensor           = torch.einsum("whadi,whjba,whkcb,whdcl->whijkl", bulk_lower[:-1,:-1],bulk_right[:-1,1:],bulk_uppe[:-1,1:],bulk_left[1:,1:])
# rigt_bulk_tensor      = torch.einsum("hadi,hjba,hkcb,hdcl->hijkl"     , rigt_lower[:-1]    ,rigt_right[1:]    ,rigt_uppe[1:]    ,bulk_left[0,1:])
# down_bulk_tensor      = torch.einsum("hadi,hjba,hkcb,hdcl->hijkl"     , down_lower[:-1]    ,bulk_right[:-1,0] ,bulk_uppe[:-1,0] ,bulk_left[1:,0])
# bulk_rigt_tensor      = torch.einsum("hadi,hjba,hkcb,hdcl->hijkl"     , bulk_lower[-1,:-1] ,bulk_right[-1,1:] ,bulk_uppe[-1,1:] ,rigt_left[1:])
# bulk_down_tensor      = torch.einsum("hadi,hjba,hkcb,hdcl->hijkl"     , bulk_lower[:-1,-1] ,down_right[:-1]   ,down_uppe[:-1]   ,down_left[1:])
# bulk_down_corn_tensor = torch.einsum("adi,jba,kcb,dcl->ijkl"          , bulk_lower[-1,-1]  ,down_right[-1]    ,down_uppe[-1]    ,corn_left)
# corn_right_bulk_tensor= torch.einsum("adi,jba,kcb,dcl->ijkl"          , corn_lower         ,rigt_right[0]     ,rigt_uppe[0]     ,bulk_left[0,0])
# right_corn_down_tensor= torch.einsum("adi,jba,kcb,dcl->ijkl"          , rigt_lower[-1]     ,corn_right        ,corn_uppe        ,down_left[0])
# down_bulk_right_tensor= torch.einsum("adi,jba,kcb,dcl->ijkl"          , down_lower[-1]     ,bulk_right[-1,0]  ,bulk_uppe[-1,0]  ,rigt_left[0])

# left,right  = apply_SVD(tensor1,left_bond=[0,1],right_bond=[2,3],truncate=truncate)# ABK,KCD
# lower,uppe  = apply_SVD(tensor2,left_bond=[1,2],right_bond=[3,0],truncate=truncate)# BCK,KDA
# new_tensor_should  = torch.einsum("whadi,whjba,whkcb,whdcl->whijkl",
#                    lower,
#                    right.roll(-1,1),
#                    uppe.roll(-1,1),
#                    left.roll(shifts=(-1, -1), dims=(0, 1))
#                    )

# print(bulk_tensor.shape)           
# print(rigt_bulk_tensor.shape)      
# print(down_bulk_tensor.shape)      
# print(bulk_rigt_tensor.shape)      
# print(bulk_down_tensor.shape)      
# print(bulk_down_corn_tensor.shape) 
# print(corn_right_bulk_tensor.shape)
# print(right_corn_down_tensor.shape)
# print(down_bulk_right_tensor.shape)

# bulk_tensor_should            = new_tensor_should[:-2, :-2]
# rigt_bulk_tensor_should       = new_tensor_should[-1,:-2]
# down_bulk_tensor_should       = new_tensor_should[:-2,-1]
# bulk_rigt_tensor_should       = new_tensor_should[-2,:-2]
# bulk_down_tensor_should       = new_tensor_should[:-2,-2]
# bulk_down_corn_tensor_should  = new_tensor_should[-2,-2]
# corn_right_bulk_tensor_should = new_tensor_should[-1,-1]
# right_corn_down_tensor_should = new_tensor_should[-1,-2]
# down_bulk_right_tensor_should = new_tensor_should[-2,-1]

# print(bulk_tensor_should.shape           )
# print(rigt_bulk_tensor_should.shape      )
# print(down_bulk_tensor_should.shape      )
# print(bulk_rigt_tensor_should.shape      )
# print(bulk_down_tensor_should.shape      )
# print(bulk_down_corn_tensor_should.shape )
# print(corn_right_bulk_tensor_should.shape)
# print(right_corn_down_tensor_should.shape)
# print(down_bulk_right_tensor_should.shape)

# print(torch.dist(bulk_tensor_should            ,bulk_tensor           ))
# print(torch.dist(rigt_bulk_tensor_should       ,rigt_bulk_tensor      ))
# print(torch.dist(down_bulk_tensor_should       ,down_bulk_tensor      ))
# print(torch.dist(bulk_rigt_tensor_should       ,bulk_rigt_tensor      ))
# print(torch.dist(bulk_down_tensor_should       ,bulk_down_tensor      ))
# print(torch.dist(bulk_down_corn_tensor_should  ,bulk_down_corn_tensor ))
# print(torch.dist(corn_right_bulk_tensor_should ,corn_right_bulk_tensor))
# print(torch.dist(right_corn_down_tensor_should ,right_corn_down_tensor))
# print(torch.dist(down_bulk_right_tensor_should ,down_bulk_right_tensor))

# new_tensor = torch.cat(
#     [torch.cat([bulk_tensor,bulk_rigt_tensor.unsqueeze(0),rigt_bulk_tensor.unsqueeze(0)]),
#      torch.cat([bulk_down_tensor,bulk_down_corn_tensor.unsqueeze(0),right_corn_down_tensor.unsqueeze(0)]).unsqueeze(1),
#      torch.cat([down_bulk_tensor,down_bulk_right_tensor.unsqueeze(0),corn_right_bulk_tensor.unsqueeze(0)]).unsqueeze(1),
#     ],dim=1)

Overwriting Contractor2D/check_the_brdc_coding_on_even_number.py


#### odd number

In [56]:
# #%%writefile Contractor2D/test_for_TRG_odd_number.py
# import torch

# from Contractor2D.utils import apply_SVD

# tensor = torch.randn(5,5,2,2,2,2)
# print(f"the input tensor network store in {tensor.shape}")
# print(f"the ultimate result should be {uniform_shape_tensor_contractor_tn(tensor)}")


# right_edge1,right_edge2 = tensor[-2:]
# right_edge = torch.einsum("wabcd,wedfg->waebcfg",right_edge1,right_edge2).flatten(1,2).flatten(-3,-2)
# down_edge1, down_edge2 = tensor[:-2,-2:].transpose(1,0)
# corn_tensor1,corn_tensor2= right_edge[-2:]
# bulk_tensor= tensor[:-2,:-2]
# right_edge = right_edge[:-2]
# down_edge  = torch.einsum("wabcd,wcefg->wabefdg",down_edge1,down_edge2).flatten(2,3).flatten(-2,-1)
# corn_tensor= torch.einsum("abcd,cefg->abefdg",corn_tensor1,corn_tensor2).flatten(1,2).flatten(-2,-1)

# print(f"the input bulk_right_left_corner tensor :")
# print(f"bulk_tensor.shape = {bulk_tensor.shape}")
# print(f"right_edge.shape  = {right_edge.shape }")
# print(f"down_edge.shape   = {down_edge.shape  }")
# print(f"corn_tensor.shape = {corn_tensor.shape}")

# print(f"the bulk_right_left_corner_contractor result {bulk_right_left_corner_contractor_tn(bulk_tensor,right_edge,down_edge,corn_tensor)}")

# lu_lu,lu_rd = apply_SVD(bulk_tensor[0::2,0::2],left_bond=[0,1],right_bond=[2,3],truncate=None)# abk ,kcd
# rd_lu,rd_rd = apply_SVD(bulk_tensor[1::2,1::2],left_bond=[0,1],right_bond=[2,3],truncate=None)# abk, kcd
# ld_ld,ld_ru = apply_SVD(bulk_tensor[0::2,1::2],left_bond=[1,2],right_bond=[3,0],truncate=None)# bck, kda
# ru_ld,ru_ru = apply_SVD(bulk_tensor[1::2,0::2],left_bond=[1,2],right_bond=[3,0],truncate=None)# bck, kda

# rd_lu_r,rd_rd_r = apply_SVD(right_edge[1::2],left_bond=[0,1],right_bond=[2,3],truncate=None)# abk, kcd
# ru_ld_r,ru_ru_r = apply_SVD(right_edge[0::2],left_bond=[1,2],right_bond=[3,0],truncate=None)# bck, kda
# rd_lu_d,rd_rd_d = apply_SVD( down_edge[1::2],left_bond=[0,1],right_bond=[2,3],truncate=None)# abk, kcd
# ld_ld_d,ld_ru_d = apply_SVD( down_edge[0::2],left_bond=[1,2],right_bond=[3,0],truncate=None)# bck, kda
# rd_lu_c,rd_rd_c = apply_SVD(corn_tensor     ,left_bond=[0,1],right_bond=[2,3],truncate=None)# bck, kda

# bulk_tensor1 = torch.einsum("whicd,whjac,whbak,whdbl->whijkl",lu_rd[:-1,:-1],ld_ru[:-1],rd_lu,ru_ld[:,:-1])
# rigt_tensor1 = torch.einsum("wicd,wjac,wbak,wdbl->wijkl",lu_rd[-1,:-1],ld_ru[-1]   ,rd_lu_r, ru_ld_r[:-1])
# down_tensor1 = torch.einsum("wicd,wjac,wbak,wdbl->wijkl",lu_rd[:-1,-1],ld_ru_d[:-1],rd_lu_d, ru_ld[:,-1])
# corn_tensor1 = torch.einsum(" icd, jac, bak, dbl-> ijkl",lu_rd[-1,-1] ,ld_ru_d[-1] ,rd_lu_c, ru_ld_r[-1])
# bulk_tensor2 = torch.einsum("whicd,whjac,whbak,whdbl->whijkl",rd_rd,ru_ru[:,1:],lu_lu[1:,1:],ld_ld[1:])
# rigt_tensor2 = torch.einsum("wicd,wjac,wbak,wdbl->wijkl",rd_rd_r, ru_ru_r[1:],lu_lu[0,1:], ld_ld[0])
# down_tensor2 = torch.einsum("wicd,wjac,wbak,wdbl->wijkl",rd_rd_d, ru_ru[:,0],lu_lu[1:,0], ld_ld_d[1:])
# corn_tensor2 = torch.einsum(" icd, jac, bak, dbl-> ijkl",rd_rd_c, ru_ru_r[0],lu_lu[0,0] , ld_ld_d[0])

# print("phase1:result")
# print(f"bulk_tensor1.shape={bulk_tensor1.shape}")
# print(f"rigt_tensor1.shape={rigt_tensor1.shape}")
# print(f"down_tensor1.shape={down_tensor1.shape}")
# print(f"corn_tensor1.shape={corn_tensor1.shape}")
# print()
# print(f"bulk_tensor2.shape={bulk_tensor2.shape}")
# print(f"rigt_tensor2.shape={rigt_tensor2.shape}")
# print(f"down_tensor2.shape={down_tensor2.shape}")
# print(f"corn_tensor2.shape={corn_tensor2.shape}")

# # if we don't truncate at this step, the map will become too complex and can not code as unique way,
# # so in such case, we will require truncate 16 in here and the recommend input not big than 32.
# truncate = None
# bulk_left ,bulk_right= apply_SVD(bulk_tensor1,left_bond=[0,1],right_bond=[2,3],truncate=truncate)
# rigt_left ,rigt_right= apply_SVD(rigt_tensor1,left_bond=[0,1],right_bond=[2,3],truncate=truncate)
# down_left ,down_right= apply_SVD(down_tensor1,left_bond=[0,1],right_bond=[2,3],truncate=truncate)
# corn_left ,corn_right= apply_SVD(corn_tensor1,left_bond=[0,1],right_bond=[2,3],truncate=truncate)
# bulk_lower,bulk_uppe = apply_SVD(bulk_tensor2,left_bond=[1,2],right_bond=[3,0],truncate=truncate)# BCK,KDA
# rigt_lower,rigt_uppe = apply_SVD(rigt_tensor2,left_bond=[1,2],right_bond=[3,0],truncate=truncate)# BCK,KDA
# down_lower,down_uppe = apply_SVD(down_tensor2,left_bond=[1,2],right_bond=[3,0],truncate=truncate)# BCK,KDA
# corn_lower,corn_uppe = apply_SVD(corn_tensor2,left_bond=[1,2],right_bond=[3,0],truncate=truncate)# BCK,KDA

# bulk_tensor_cent      = torch.einsum("whadi,whjba,whkcb,whdcl->whijkl", bulk_lower[:-1,:-1],bulk_right[:-1,1:],bulk_uppe[:-1,1:],bulk_left[1:,1:])
# rigt_bulk_tensor      = torch.einsum("hadi,hjba,hkcb,hdcl->hijkl"     , rigt_lower[:-1]    ,rigt_right[1:]    ,rigt_uppe[1:]    ,bulk_left[0,1:])
# down_bulk_tensor      = torch.einsum("hadi,hjba,hkcb,hdcl->hijkl"     , down_lower[:-1]    ,bulk_right[:-1,0] ,bulk_uppe[:-1,0] ,bulk_left[1:,0])
# bulk_rigt_tensor      = torch.einsum("hadi,hjba,hkcb,hdcl->hijkl"     , bulk_lower[-1,:-1] ,bulk_right[-1,1:] ,bulk_uppe[-1,1:] ,rigt_left[1:])
# bulk_down_tensor      = torch.einsum("hadi,hjba,hkcb,hdcl->hijkl"     , bulk_lower[:-1,-1] ,down_right[:-1]   ,down_uppe[:-1]   ,down_left[1:])
# bulk_down_corn_tensor = torch.einsum("adi,jba,kcb,dcl->ijkl"          , bulk_lower[-1,-1]  ,down_right[-1]    ,down_uppe[-1]    ,corn_left)
# corn_right_bulk_tensor= torch.einsum("adi,jba,kcb,dcl->ijkl"          , corn_lower         ,rigt_right[0]     ,rigt_uppe[0]     ,bulk_left[0,0])
# right_corn_down_tensor= torch.einsum("adi,jba,kcb,dcl->ijkl"          , rigt_lower[-1]     ,corn_right        ,corn_uppe        ,down_left[0])
# down_bulk_right_tensor= torch.einsum("adi,jba,kcb,dcl->ijkl"          , down_lower[-1]     ,bulk_right[-1,0]  ,bulk_uppe[-1,0]  ,rigt_left[0])
# # down_bulk_tensor       = down_bulk_tensor[None]
# # down_bulk_right_tensor = down_bulk_right_tensor[None]
# # corn_right_bulk_tensor = corn_right_bulk_tensor[None]
# # bulk_down_corn_tensor  = bulk_down_corn_tensor[None]

# print("phase2 result:")
# print(f"bulk_tensor_cent.shape      ={bulk_tensor_cent.shape      }")
# print(f"down_bulk_tensor.shape      ={down_bulk_tensor.shape      }")      
# print(f"bulk_rigt_tensor.shape      ={bulk_rigt_tensor.shape      }")   
# print(f"down_bulk_right_tensor.shape={down_bulk_right_tensor.shape}")
# print(f"rigt_bulk_tensor.shape      ={rigt_bulk_tensor.shape      }")      
# print(f"corn_right_bulk_tensor.shape={corn_right_bulk_tensor.shape}")
# print(f"bulk_down_tensor.shape      ={bulk_down_tensor.shape      }") 
# print(f"bulk_down_corn_tensor.shape ={bulk_down_corn_tensor.shape }") 
# print(f"right_corn_down_tensor.shape={right_corn_down_tensor.shape}")

# bulk_tensor = torch.cat([
#     torch.cat([bulk_tensor_cent,down_bulk_tensor[None]],1),
#     torch.cat([bulk_rigt_tensor,down_bulk_right_tensor[None]])[None]
# ])

# bulk_tensor.shape

# bulk_tensor = down_bulk_right_tensor[None][None]

# right_edge = torch.cat([rigt_bulk_tensor,corn_right_bulk_tensor[None]])
# down_edge  = torch.cat([bulk_down_tensor,bulk_down_corn_tensor[None]])
# corn_tensor =right_corn_down_tensor

# print(f"next tensor shape:")
# print(f"bulk_tensor.shape = {bulk_tensor.shape}")
# print(f"right_edge.shape  = {right_edge.shape }")
# print(f"down_edge.shape   = {down_edge.shape  }")
# print(f"corn_tensor.shape = {corn_tensor.shape}")

# print(f"next tensor contraction result:")
# print(bulk_right_left_corner_contractor_tn(bulk_tensor,right_edge,down_edge,corn_tensor))

#### time test for different engine (only bulk $2^n$)

In [20]:
# import torch_semiring_einsum
# def TRT_semiring(tensor,truncate=None, block_size=5):
#     W,H = tensor.shape[:2]
#     einsum_1 = torch_semiring_einsum.compile_equation("whicd,whjac,whbak,whdbl->whijkl")
#     einsum_2 = torch_semiring_einsum.compile_equation("whadi,whjba,whkcb,whdcl->whijkl")
#     einsum_3 = torch_semiring_einsum.compile_equation("abcd,ciaj,ldkb,kjli->")
#     while True:
#         print(tensor.shape)
#         W,H = tensor.shape[:2]
#         #if W<=1 or H<=1:break
#         lu_tensor = tensor[0::2,0::2]
#         ld_tensor = tensor[0::2,1::2]
#         ru_tensor = tensor[1::2,0::2]
#         rd_tensor = tensor[1::2,1::2]
#         if W<=2 or H<=2:break
            
#         lu_lu,lu_rd = apply_SVD(lu_tensor,left_bond=[0,1],right_bond=[2,3],truncate=truncate)# abk ,kcd
#         rd_lu,rd_rd = apply_SVD(rd_tensor,left_bond=[0,1],right_bond=[2,3],truncate=truncate)# abk, kcd
#         ld_ld,ld_ru = apply_SVD(ld_tensor,left_bond=[1,2],right_bond=[3,0],truncate=truncate)# bck, kda
#         ru_ld,ru_ru = apply_SVD(ru_tensor,left_bond=[1,2],right_bond=[3,0],truncate=truncate)# bck, kda
#         tensor1     = torch_semiring_einsum.einsum(einsum_1,
#                                     lu_rd,
#                                     ld_ru,
#                                     rd_lu,
#                                     ru_ld,block_size=block_size)
#         tensor2     = torch_semiring_einsum.einsum(einsum_1,
#                                    rd_rd,
#                                    ru_ru.roll(shifts=-1,dims=1),
#                                    lu_lu.roll(shifts=(-1, -1), dims=(0, 1)),
#                                    ld_ld.roll(shifts=-1,dims=0),block_size=block_size)
#         left,right  = apply_SVD(tensor1,left_bond=[0,1],right_bond=[2,3],truncate=truncate)# ABK,KCD
#         lower,uppe  = apply_SVD(tensor2,left_bond=[1,2],right_bond=[3,0],truncate=truncate)# BCK,KDA
#         tensor  = torch_semiring_einsum.einsum(einsum_2,
#                            lower,
#                            right.roll(-1,1),
#                            uppe.roll(-1,1),
#                            left.roll(shifts=(-1, -1), dims=(0, 1)),block_size=block_size
#                            )
#     value   = torch_semiring_einsum.einsum(einsum_3,
#                            lu_tensor[0,0],ld_tensor[0,0],
#                            ru_tensor[0,0],rd_tensor[0,0],block_size=block_size)
#     #value = torch.einsum("abab->",tensor[0,0])
#     return value

In [ ]:
tensor = torch.randn(14,14,2,2,2,2)/2

In [4]:
def TRT(tensor,truncate=None,einsum_engin=torch.einsum):
    W,H = tensor.shape[:2]
    while True:
        print(tensor.shape)
        W,H = tensor.shape[:2]
        #if W<=1 or H<=1:break
        lu_tensor = tensor[0::2,0::2]
        ld_tensor = tensor[0::2,1::2]
        ru_tensor = tensor[1::2,0::2]
        rd_tensor = tensor[1::2,1::2]
        if W<=2 or H<=2:break
#         print("SVD:time"),
#         start_time  = time.time()
        lu_lu,lu_rd = apply_SVD(lu_tensor,left_bond=[0,1],right_bond=[2,3],truncate=truncate)# abk ,kcd
        rd_lu,rd_rd = apply_SVD(rd_tensor,left_bond=[0,1],right_bond=[2,3],truncate=truncate)# abk, kcd
        ld_ld,ld_ru = apply_SVD(ld_tensor,left_bond=[1,2],right_bond=[3,0],truncate=truncate)# bck, kda
        ru_ld,ru_ru = apply_SVD(ru_tensor,left_bond=[1,2],right_bond=[3,0],truncate=truncate)# bck, kda
#         cost = time.time() - start_time
#         print(cost)       
#         print("einsum:time"),
#         start_time  = time.time()
        tensor1     = einsum_engin("whicd,whjac,whbak,whdbl->whijkl",
                                    lu_rd,
                                    ld_ru,
                                    rd_lu,
                                    ru_ld)
        tensor2     = einsum_engin("whicd,whjac,whbak,whdbl->whijkl",
                                   rd_rd,
                                   ru_ru.roll(shifts=-1,dims=1),
                                   lu_lu.roll(shifts=(-1, -1), dims=(0, 1)),
                                   ld_ld.roll(shifts=-1,dims=0))
#         cost = time.time() - start_time
#         print(cost)
#         print("SVD:time"),
#         start_time  = time.time()
        
        left,right  = apply_SVD(tensor1,left_bond=[0,1],right_bond=[2,3],truncate=truncate)# ABK,KCD
        lower,uppe  = apply_SVD(tensor2,left_bond=[1,2],right_bond=[3,0],truncate=truncate)# BCK,KDA
#         cost = time.time() - start_time
#         print(cost)
#         print("einsum:time"),
#         start_time  = time.time()
        tensor  = einsum_engin("whadi,whjba,whkcb,whdcl->whijkl",
                           lower,
                           right.roll(-1,1),
                           uppe.roll(-1,1),
                           left.roll(shifts=(-1, -1), dims=(0, 1))
                           )
#         cost = time.time() - start_time
#         print(cost)
    value   = einsum_engin("abcd,ciaj,ldkb,kjli->",lu_tensor[0,0],ld_tensor[0,0],ru_tensor[0,0],rd_tensor[0,0])
    #value = torch.einsum("abab->",tensor[0,0])
    return value

In [ ]:
tensor = torch.randn(14,14,2,2,2,2)/2

In [4]:
engine=lambda equ,*karg:torch_semiring_einsum.einsum(torch_semiring_einsum.compile_equation(equ),*karg,block_size=10)

In [24]:
tensor = torch.randn(4,4,2,2,2,2)
%timeit TRT(tensor,truncate=None)

2.49 ms ± 10 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [23]:
%timeit TRT(tensor,truncate=None,einsum_engin=contract)

3.67 ms ± 1.43 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### Boundary MPS

In [65]:
import numpy as np
import torch
import time
from tqdm.notebook import trange, tqdm
from utils import *


def rd_engine(*x,**kargs):
    x =  torch.randn(*x,device='cpu',**kargs)
    x/=  torch.norm(x).sqrt()
    return x

In [3]:
D=4;P=4;L=7;
# top_mps_list    = [rd_engine(P,D)] + [rd_engine(L-2,D,P,D)]  + [rd_engine(D,P)]
# middle_mpo_list = [[rd_engine(P,D,P)]+[rd_engine(L-2,D,P,D,P)]+ [rd_engine(D,P,P)]
#                   for _ in range(L-2)]
# bottom_mps_list = [rd_engine(P,D)] + [rd_engine(L-2,D,P,D)]  + [rd_engine(D,P)]
# peps  = [top_mps_list]+middle_mpo_list+[bottom_mps_list]

In [2]:
D=4;P=4;L=7;
top_mps_list    = [rd_engine(D,D)]     + [rd_engine(L-2,D,D,D)]  + [rd_engine(D,D)]
middle_mpo_list = [[rd_engine(D,D,D)]  + [rd_engine(L-2,D,D,D,D)]+ [rd_engine(D,D,D)]
                  for _ in range(L-2)]
bottom_mps_list = [rd_engine(D,D)]     + [rd_engine(L-2,D,D,D)]  + [rd_engine(D,D)]
peps  = [top_mps_list]+middle_mpo_list+[bottom_mps_list]

In [13]:
# from engine.torch_dense import approxmate_mps_line
import torch
import numpy as np
def truncated_SVD(tensor,output='RQ',max_singular_values= None,
                  max_truncation_error= None,
                  relative = True,
                  normlized= True,
                  verbose  = False,auto_check_diagonal=False):
    # the canonocal
    # tensor is batched
    reduce = False
    u=s=v = None
    if len(tensor.shape)==2:
        tensor = tensor.unsqueeze(0)
        reduce = True
    if auto_check_diagonal:
        out = diagonal_tensor_svd_torch_dense(tensor)
        if out is not None:u, s, v = out
    if u is None: u, s, v = torch.svd(tensor)


    max_singular_values = s.shape[-1] if max_singular_values is None else max_singular_values

    if max_truncation_error is not None:
        # Cumulative norms of singular values in ascending order
        s_sorted, _ = torch.sort(s**2,-1)
        trunc_errs  = torch.sqrt(torch.cumsum(s_sorted, -1))
        # If relative is true, rescale max_truncation error with the largest
        # singular value to yield the absolute maximal truncation error.
        abs_max_truncation_error = max_truncation_error * s[:,0:1] if relative else max_truncation_error
        # We must keep at least this many singular values to ensure the
        # truncation error is <= abs_max_truncation_error.
        num_sing_vals_err = torch.sum(trunc_errs > abs_max_truncation_error,-1).max()
        if max_singular_values>num_sing_vals_err and verbose:
            print(f"use {num_sing_vals_err}/{max_singular_values} sing vals")
    else:
        num_sing_vals_err  = max_singular_values

    num_sing_vals_keep = min(max_singular_values, num_sing_vals_err)


    #s_rest = s[...,num_sing_vals_keep:]
    u      = u[...,:num_sing_vals_keep]
    s      = s[...,:num_sing_vals_keep]
    v      = v[...,:num_sing_vals_keep]
    v      = torch.transpose(v, -1, -2)#vh

    if num_sing_vals_keep == s.shape[-1] and normlized:
        Z = 1.0*torch.ones(s.shape[0])
    else:
        Z = torch.sum(s**2,-1).sqrt()

    if output == 'RQ':
        R = torch.einsum('iab,ibc->iac',u ,torch.diag_embed(s))
        Q = v
        output = [R,Q,Z]
    elif output == 'QR':
        Q = u
        R = torch.einsum('iab,ibc->iac',torch.diag_embed(s),v)
        output = [Q,R,Z]
    else:
        output = [u,s,v,Z]
    if reduce:output = [t[0] for t in output]
    return output
def left_canonicalize_MPS(mps_line,Decomposition_Engine=torch.qr,
                          normlization =True):
    # for any not canonical mps line
    # the chain size (D,P,D)
    new_chain = []
    R         = None
    #Z_list    = []# record the scale information for each unit.
    # for a perfect MPS state, we expect the norm for each tensor is 1.
    for i,tensor in enumerate(mps_line):
        if len(tensor.shape)==2:
            new_tensor = torch.einsum('ab,bd->ad',R,tensor) if R is not None else tensor
            shape      = new_tensor.shape
        elif len(tensor.shape)==3:
            new_tensor = torch.einsum('ab,bcd->acd',R,tensor) if R is not None else tensor
            shape      = new_tensor.shape
            a,b,c = shape
            new_tensor = new_tensor.reshape(a*b,c)
        else:
            raise NotImplementedError

        if i == len(mps_line) - 1:
            Z = torch.norm(new_tensor)
            if normlization:new_tensor /= (Z)
            new_chain.append(new_tensor.reshape(*shape[:-1],-1))
        else:
            Q,R = Decomposition_Engine(new_tensor)[:2]
            Q   = Q.reshape(*shape[:-1],-1)
            new_chain.append(Q)

    return new_chain,[Z]
def right_canonicalize_MPS(mps_line,Decomposition_Engine=truncated_SVD,
                          #normlization =True
                          ):
    new_chain = []
    R         = None
    Z_list    = []
    svd_Z         = torch.Tensor([1.0])#input has already been normalized
    for i,tensor in enumerate(mps_line[::-1]):

        if len(tensor.shape)==2:
            new_tensor = torch.einsum('ab,bc->ac',tensor, R) if R is not None else tensor
            shape      = new_tensor.shape
        elif len(tensor.shape)==3:
            new_tensor = torch.einsum('alb,bc->alc',tensor, R) if R is not None else tensor
            shape      = new_tensor.shape
            a,b,c      = shape
            new_tensor = new_tensor.reshape(a,b*c)
        else:
            raise NotImplementedError
        Z = torch.norm(new_tensor)
        Z_list.append(Z)
        new_tensor /= Z
        # normlization is necessary; directly use the SVD_Z may cause problem due to precision
        #print(f"svd_Z{svd_Z.item()}<->all_Z {Z.item()} <-> after_Z{torch.norm(new_tensor).item()}")
        #if normlization:new_tensor /= Z
        if i == len(mps_line) - 1:
            new_chain.append(new_tensor.reshape(-1,*shape[1:]))
        else:
            R,Q,svd_Z = Decomposition_Engine(new_tensor)
            Q   = Q.reshape(-1,*shape[1:])
            new_chain.append(Q)
    new_chain=new_chain[::-1]
    return new_chain,Z_list
def torchrq(tensor):
    q, r = torch.qr(torch.transpose(tensor, -2, -1))
    r, q = torch.transpose(r, -2, -1), torch.transpose(q, -2, -1)  #M=r*q at this point
    return r,qx
def approxmate_mps_line(mps_line,
                        max_singular_values= None,max_truncation_error= None,relative = True,
                        mode='full',left_method='qr'
                       ):


    scalar = 1
    if mode != 'right':
        if left_method == 'qr':
            DCEngine = torch.linalg.qr if float(torch.__version__[:4])>1.07 else torch.qr
        elif left_method == 'svd':
            DCEngine = lambda x:truncated_SVD(x,output='QR')
        else:
            raise NotImplementedError
        mps_line,Z_list = left_canonicalize_MPS(mps_line,Decomposition_Engine=DCEngine)
        #print(get_mps_size_list(mps_line))
        print(f"   left canonical scalar:{np.prod(Z_list)}")
        scalar *= np.prod(Z_list)
        #print(f"now tensor norm: {torch.norm(mps_line[-1])}")
    SVD_Engine = lambda x:truncated_SVD(x,max_singular_values = max_singular_values,
                                          max_truncation_error= max_truncation_error,
                                          relative = relative)
    mps_line,Z_list = right_canonicalize_MPS(mps_line,Decomposition_Engine=SVD_Engine)
    #print(get_mps_size_list(mps_line))
    #print(f"   right canonical Z:{[np.round(t.item(),3) for t in Z_list]}")
    print(f"   right canonical scalar:{np.prod(Z_list)}")
    scalar *= np.prod(Z_list)
    return mps_line,scalar
def diagonal_tensor_svd_torch_dense(tensor):
    # support batch tensor
    reduce = False
    if len(tensor.shape)==2:
        tensor = tensor.unsqueeze(0)
        reduce = True
    W,H   = tensor.shape[-2:]
    batch_shape= tensor.shape[:-2]
    u = s = v = None
    if W>=H:
        batch_diag = torch.matmul(tensor.transpose(-1,-2),tensor)#auto broadcast, or can use bmm
    else:
        batch_diag = torch.matmul(tensor,tensor.transpose(-1,-2))#auto broadcast, or can use bmm
    diagnol_num = torch.diagonal(a,dim1=-2,dim2=-1).nelement()
    tensor_num  = tensor.nelement()
    if diagnol_num != tensor_num:return None
    A,A        = batch_diag.shape[-2:]
    batch_diag = batch_diag[...,range(A),range(A)]
    batch_diag,batch_order= batch_diag.sort(-1,descending=True)
    #fast_V      = [get_sort_matrix(order).to_dense() for order in batch_order]
    batch_order = batch_order.flatten(start_dim=0,end_dim=-2)
    K,A         = batch_order.shape
    s           = batch_diag.sqrt()
    s           = s.reshape(*batch_shape,A)
    if W>=H:
        v = torch.sparse_coo_tensor([list(range(K*A)),batch_order.flatten().tolist()], [1.0]*K*A,(K*A,A))
        v = v.to_dense().reshape(-1,A,A)
        u = torch.bmm(tensor,v.transpose(-1,-2)/s.unsqueeze(-2))
        v = v.reshape(*batch_shape,A,A)
        u = u.reshape(*batch_shape,W,A)
    else:
        u = torch.sparse_coo_tensor([list(range(K*A)),batch_order.flatten().tolist()], [1.0]*K*A,(K*A,A))
        u = u.to_dense().reshape(-1,A,A).transpose(-1,-2)
        v = torch.bmm(u.transpose(-1,-2)/s.unsqueeze(-1),tensor)#TODO: case when s==0
        u = u.reshape(*batch_shape,A,A)
        v = v.reshape(*batch_shape,A,H)
    output = [u,s,v.transpose(-1,-2)]
    if reduce:
        output = [t[0] for t in output]
    return output

In [8]:
torch.__version__

'1.10.0+cu102'

In [18]:
tensor = peps[0]
for mpo in peps[1:-1]:
    tensor = contract_mps_mpo(tensor,mpo)
    #print(get_mps_size_list(tensor))
    tensor = right_mps_form(tensor)
    tensor,scale = approxmate_mps_line(tensor,max_singular_values=100)
    print(get_mps_size_list(tensor))
tensor = contract_two_mps_tn(tensor,peps[-1])

   left canonical scalar:0.5324269533157349
   right canonical scalar:0.9999996423721313
(4, 4)-(4, 4, 16)- 3x(16, 4, 16) -(16, 4, 4)-(4, 4)
   left canonical scalar:0.17249836027622223
   right canonical scalar:0.9999997615814209
(4, 4)-(4, 4, 16)-(16, 4, 64)-(64, 4, 64)-(64, 4, 16)-(16, 4, 4)-(4, 4)
   left canonical scalar:0.141238734126091
   right canonical scalar:0.9999996423721313
(4, 4)-(4, 4, 16)-(16, 4, 64)-(64, 4, 64)-(64, 4, 16)-(16, 4, 4)-(4, 4)
   left canonical scalar:0.13460852205753326
   right canonical scalar:0.9999997019767761
(4, 4)-(4, 4, 16)-(16, 4, 64)-(64, 4, 64)-(64, 4, 16)-(16, 4, 4)-(4, 4)
   left canonical scalar:0.1684599667787552
   right canonical scalar:1.000000238418579
(4, 4)-(4, 4, 16)-(16, 4, 64)-(64, 4, 64)-(64, 4, 16)-(16, 4, 4)-(4, 4)


In [3]:
def onebyoneBMPS(tensor,truncate=None,einsum_engin=torch.einsum):
    W,H = tensor.shape[:2]
    while W > 1:
            half_size = size // 2
            nice_size = 2 * half_size
            leftover  = tensor[nice_size:]
            tensor    = torch.einsum("mbik,mbkj->mbij",tensor[0:nice_size:2], tensor[1:nice_size:2])
            #(k/2,NB,D,D),(k/2,NB,D,D) <-> (k/2,NB,D,D)
            tensor   = torch.cat([tensor, leftover], axis=0)
            size     = half_size + int(size % 2 == 1)
    return value